In [27]:
from bs4 import BeautifulSoup
import re, time
import datetime, sqlite3
import urllib.request
from io import BytesIO
from PIL import Image
from IPython.display import Image as IMG, display 

def get(db_name,data_name):
    conn = sqlite3.connect(db_name)
    rows=[]
    for row in conn.execute("select * from "+data_name).fetchall():
        rows.append(row)
    conn.close()
    return rows
    
def run(db_name,sql):
    conn = sqlite3.connect(db_name)
    for row in conn.execute(sql):
        print (row)
    conn.close()
    
def getInsertedCarData():
    db_name="myown.db"
    data_name="car_images"
    cars=get(db_name, data_name)
    return cars

def getInsertedCarLinks():
    inserted=getInsertedCarData()
    links=[]
    for i in inserted:
        links.append(i[1])
    return links

class AppURLopener(urllib.request.FancyURLopener):
    version = "Mozilla/5.0"

def getImagesFromNettiautoLink(link):
    opener = AppURLopener()
    response = opener.open(link).read()
    soup = BeautifulSoup(response, "lxml")
    dataHtml = soup.findAll("link")
    
    images=[]
    for d in dataHtml:
        if(d["href"].endswith("jpg")):
            URL=d["href"]
            #print(d)
            file = BytesIO(urllib.request.urlopen(URL).read())
            img = Image.open(file)
            images.append(img)
    
    return images

def getImagesFromAutotieLink(link):
    opener = AppURLopener()
    response = opener.open(link).read()
    soup = BeautifulSoup(response, "lxml")
    dataHtml = soup.findAll("img")
    images=[]
    failed=0
    for d in dataHtml:
        try:
            if "www.autotie.fi" in d["src"]:
                URL="https://"+d["src"][2:]
                file = BytesIO(urllib.request.urlopen(URL).read())
                img = Image.open(file)
                images.append(img)
        except KeyboardInterrupt:
            raise
        except:
            failed=failed+1#how many urls dont have proper url
            #raise
            #print("no link for src")
    return images

        
def getImagesFromAutotalliLink(link):
    opener = AppURLopener()
    response = opener.open(link).read()
    soup = BeautifulSoup(response, "lxml")
    dataHtml = soup.findAll("img")
    #print(soup)
    #print(dataHtml)
    #return 0
    images=[]
    name=""
    for d in dataHtml:
        if(d["src"].endswith("original.jpg") and "800x600" in d["src"]):
            name=d["alt"]
            URL=d["src"]
            #print(name, URL)
            file = BytesIO(urllib.request.urlopen(URL).read())
            %matplotlib inline
            img = Image.open(file)
            images.append(img)
            return images #no need to get multiple images
    return images

def insertDB(db_name, data_name, data, date='', filename=''):
    conn = sqlite3.connect(db_name)
    c = conn.cursor()
    if date == '':
        now = datetime.datetime.now()
        date=now.strftime("%Y-%m-%d")

    if data_name == "cars_raw":
        #print("Cars data filename is ", filename)
        # Create table
        c.execute("""CREATE TABLE IF NOT EXISTS cars_raw (ind integer, link text, name text, price integer, kms integer, year integer, insert_date date, batchname varchar(50))""")
        
        count=conn.execute("select count(*) from cars_raw")
        for cou in count:
            count=cou[0]
        # Larger example that inserts many records at a time
        data_insert=[]
        for d in data:
            #Generate the rows
            count = count+1
            data_insert.append([count, d[0], d[1], d[2], d[3], d[4], date, filename])
        c.executemany('INSERT INTO cars_raw VALUES (?,?,?,?,?,?,?,?)', data_insert)
    
    elif data_name == "car_images":
        #print("Cars data filename is ", filename)
        # Create table
        c.execute("""CREATE TABLE IF NOT EXISTS car_images (ind integer, link text, name text, path text, insert_date date)""")
        
        count=conn.execute("select count(*) from car_images")
        for cou in count:
            count=cou[0]
        # Larger example that inserts many records at a time
        data_insert=[]
        #print(data)
        for d in data:
            #print(d)
            #Generate the rows
            count = count+1
            data_insert.append([count, d[0], d[1], d[2], date])
        c.executemany('INSERT INTO car_images VALUES (?,?,?,?,?)', data_insert)

    elif data_name == "test":
        count=conn.execute("select count(*) from "+data_name)
        for cou in count:
            count=cou[0]
        # Larger example that inserts many records at a time
        data_insert=[]
        for d in data:
            #Generate the rows
            count = count+1
            data_insert.append([count, d[0], d[1], d[2], d[3], d[4], date])
        c.executemany('INSERT INTO ' + data_name + ' VALUES (?,?,?,?,?,?,?)', data_insert)

    # Save (commit) the changes
    conn.commit()
    # Close the connection
    conn.close()

In [28]:
#test
'''nettiauto_link="https://www.nettiauto.com/renault/laguna/9884377"
autotie_link="https://www.autotie.fi/vaihtoautot/Mercedes-Benz/E/CXyFhn8TbaipbZbwX27Brg%3d%3d"
autotalli_link="https://www.autotalli.com/vaihtoauto/36270552/Ford/Transit/2015/Tampere?searchType=usedCar"

i1=getImagesFromNettiautoLink(nettiauto_link)
i2=getImagesFromAutotieLink(autotie_link)
i3=getImagesFromAutotalliLink(autotalli_link)

display(i1[0])
display(i2[0])
display(i3[0])'''


'nettiauto_link="https://www.nettiauto.com/renault/laguna/9884377"\nautotie_link="https://www.autotie.fi/vaihtoautot/Mercedes-Benz/E/CXyFhn8TbaipbZbwX27Brg%3d%3d"\nautotalli_link="https://www.autotalli.com/vaihtoauto/36270552/Ford/Transit/2015/Tampere?searchType=usedCar"\n\ni1=getImagesFromNettiautoLink(nettiauto_link)\ni2=getImagesFromAutotieLink(autotie_link)\ni3=getImagesFromAutotalliLink(autotalli_link)\n\ndisplay(i1[0])\ndisplay(i2[0])\ndisplay(i3[0])'

In [39]:
#get data
import sqlite3
import datetime
db_name="myown.db"
data_name="cars_raw"

#Get new links from 3days before to this day
yesterday = str((datetime.datetime.now()- datetime.timedelta(3)).date()) 
#Get cars which have multiple values in the database
conn = sqlite3.connect(db_name)
autotalli=conn.execute("select distinct link, name from cars_raw where link like '%autotalli%' and insert_date >= '"+yesterday+"'").fetchall()
autotie=conn.execute("select distinct link, name from cars_raw where link like '%autotie%' and insert_date >= '"+yesterday+"'").fetchall()
nettiauto=conn.execute("select distinct link, name from cars_raw where link like '%nettiauto%' and insert_date >= '"+yesterday+"'").fetchall()
#ret=conn.execute("select ind, name from cars_raw").fetchall()
'''print (autotalli[0:2])
print (autotie[0:2])
print (nettiauto[0:2])'''
conn.close()

#Get already inserted links
insertedCarLinks=getInsertedCarLinks()
print(insertedCarLinks)
#print(get(db_name, data_name))
'''run(db_name, "DROP TABLE car_images")
print(get(db_name, data_name))'''

['https://www.autotie.fi/vaihtoautot/Mercedes-Benz/E/CXyFhn8TbaipbZbwX27Brg%3d%3d', 'https://www.autotie.fi/vaihtoautot/Mercedes-Benz/E/PNbUsShj9uA4WN-DS9acvg%3d%3d', 'https://www.autotie.fi/vaihtoautot/Mercedes-Benz/GLA/oPOdFDClOWtHd-jRmc0QGQ%3d%3d', 'https://www.autotie.fi/vaihtoautot/Mercedes-Benz/E/CXyFhn8TbaipbZbwX27Brg%3d%3d', 'https://www.autotie.fi/vaihtoautot/Mercedes-Benz/E/PNbUsShj9uA4WN-DS9acvg%3d%3d', 'https://www.autotie.fi/vaihtoautot/Mercedes-Benz/GLA/oPOdFDClOWtHd-jRmc0QGQ%3d%3d', 'https://www.autotie.fi/vaihtoautot/Skoda/Octavia/wVcdrcnuAE4%3d', 'https://www.autotie.fi/vaihtoautot/Volvo/XC90/ZBNmV8XVbxI%3d', 'https://www.autotie.fi/vaihtoautot/BMW/X5/Lwt8KB8k4CoW4iU_-jG8Lg%3d%3d', 'https://www.autotie.fi/vaihtoautot/Volvo/XC70/EzXCVyB1CfM%3d', 'https://www.autotie.fi/vaihtoautot/Mercedes-Benz/E/eDlNPEu3dHOApBrDkoK4cg%3d%3d', 'https://www.autotie.fi/vaihtoautot/Mercedes-Benz/A/e_OzG2g0jvg%3d', 'https://www.autotie.fi/vaihtoautot/Volvo/V60/bXZZ_gSzo0c%3d', 'https://www.

'run(db_name, "DROP TABLE car_images")\nprint(get(db_name, data_name))'

In [ ]:
#run 
import os

def makeDirIfNotExist(directory):
    if not os.path.exists(directory):
        os.makedirs(directory)
        
def countFilesInDir(directory):
    count=1
    for file in os.listdir(directory):
        if file.endswith(".jpg") or file.endswith(".png"):
            count=count+1
    return count

def saveImage(name, img):
    directory="./images/"+name+"/"
    makeDirIfNotExist(directory)
    c=countFilesInDir(directory)
    name_path=directory+"img"+str(c)+".jpg"
    img.save(name_path)
    return name_path


def getAllImages(insertedCarLinks):
    links=[]
    names=[]
    image_paths=[]
    
    print("Getting autotie images")
    for link in autotie:
        print(link[0])
        if link[0] not in insertedCarLinks:
            try:
                img=getImagesFromAutotieLink(link[0])[0]
                img_path=saveImage(link[1],img)
                image_paths.append(img_path)
                names.append(link[1])
                links.append(link[0])
                print(img_path)
            except KeyboardInterrupt:
                print("Keyboard interrupt")
                return links,names,image_paths
            except Exception as e:
                print(e)
            
    print("Getting autotalli images")
    for link in autotalli:
        if link[0] not in insertedCarLinks:
            try:
                img=getImagesFromAutotalliLink(link[0])[0]
                img_path=saveImage(link[1],img)
                image_paths.append(img_path)
                names.append(link[1])
                links.append(link[0])
                print(img_path)
            except KeyboardInterrupt:
                print("Keyboard interrupt")
                return
            except Exception as e:
                print(e)
            
    print("Getting nettiauto images")
    for link in nettiauto:
        if link[0] not in insertedCarLinks:   
            try:
                img_path=getImagesFromNettiautoLink(link[0])[0]
                img_path=saveImage(link[1],img)
                image_paths.append(img_path)
                links.append(link[0])
                print(img_path)
            except KeyboardInterrupt:
                print("Keyboard interrupt")
                return
            except Exception as e:
                print(e)
            
    return links,names,image_paths
            
links,names,paths=getAllImages(insertedCarLinks)
print("Done")
print(links,names,paths)

Getting autotie images
https://www.autotie.fi/vaihtoautot/Mercedes-Benz/E/CXyFhn8TbaipbZbwX27Brg%3d%3d
https://www.autotie.fi/vaihtoautot/Mercedes-Benz/E/PNbUsShj9uA4WN-DS9acvg%3d%3d
https://www.autotie.fi/vaihtoautot/Mercedes-Benz/GLA/oPOdFDClOWtHd-jRmc0QGQ%3d%3d
https://www.autotie.fi/vaihtoautot/Skoda/Octavia/wVcdrcnuAE4%3d
https://www.autotie.fi/vaihtoautot/Volvo/XC90/ZBNmV8XVbxI%3d
https://www.autotie.fi/vaihtoautot/BMW/X5/Lwt8KB8k4CoW4iU_-jG8Lg%3d%3d
https://www.autotie.fi/vaihtoautot/Volvo/XC70/EzXCVyB1CfM%3d
https://www.autotie.fi/vaihtoautot/Mercedes-Benz/E/eDlNPEu3dHOApBrDkoK4cg%3d%3d
https://www.autotie.fi/vaihtoautot/Mercedes-Benz/A/e_OzG2g0jvg%3d
https://www.autotie.fi/vaihtoautot/Volvo/V60/bXZZ_gSzo0c%3d
https://www.autotie.fi/vaihtoautot/Mercedes-Benz/E/RC2DUeqOH4c%3d
https://www.autotie.fi/vaihtoautot/SEAT/Altea/FJiOWV_6g9c%3d
https://www.autotie.fi/vaihtoautot/Mercedes-Benz/GLC/X6vzESIehWI%3d
https://www.autotie.fi/vaihtoautot/Volvo/V90/8YCAZz2R2QY%3d
https://www.autot

/usr/lib/python3/dist-packages/ipykernel_launcher.py:57: DeprecationWarning: AppURLopener style of invoking requests is deprecated. Use newer urlopen functions/methods


./images/Toyota Rav4/img6.jpg
https://www.autotie.fi/vaihtoautot/Audi/A6/GzRSAN5P7do%3d
./images/Audi A6/img8.jpg
https://www.autotie.fi/vaihtoautot/Volvo/V60/G3OzJlyQL1o%3d
list index out of range
https://www.autotie.fi/vaihtoautot/Volvo/V40/VRHH6mAtXc4%3d


In [38]:
db_name="myown.db"
data_name="car_images"
data=[]
for i in range(len(links)):
    data.append([links[i], names[i], paths[i]])
#print(data)
insertDB(db_name, data_name, data)
print("Done")

Done


Done
